# Sesiunea 8 – Design Patterns în Python
_Notebook de exerciții (fără soluții)._

## Ex. 1 — Test **Singleton**
- Creează o clasă `Logger` implementată ca **Singleton** (o singură instanță în tot programul).
- Metodă minimă: `log(mesaj: str)` care afișează sau reține mesajul.
- Creează două variabile `log1` și `log2` și demonstrează că referă **aceeași** instanță (ex.: `id(log1) == id(log2)`).
- Nu folosi soluții externe; implementează tu mecanismul Singleton.


In [1]:
class Logger:
    _instance = None
    _message = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def log(self, message: str = None):
        if message:
            self._message = message
        else:
            print(self._message)
    
logger1 = Logger()
logger2 = Logger()

print(logger1)
print(logger2)

logger2.log("test")
logger1.log()


test


In [2]:
# with decorator
def singleton(cls):
    instances = {}

    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]

    return get_instance

# Logger = singleton(Logger) -> get_instance
@singleton
class Logger:
    def __init__(self):
        self._message = None

    def log(self, message: str = None):
        if message:
            self._message = message
        else:
            print(self._message)


# this calls get_instance(Logger)
logger1 = Logger()
logger2 = Logger()

print(logger1)
print(logger2)

logger2.log("test")
logger1.log()

test


## Ex. 2 — Test **Factory**
- Implementează o **Factory** (de ex. `ProdusFactory`) care creează obiecte diferite în funcție de un parametru `tip` (ex.: `"telefon"`, `"laptop"`).
- Fiecare tip are o clasă cu metoda `descriere()`.
- Apelează `ProdusFactory.creeaza_produs("telefon")` și afișează descrierea.


In [3]:
@singleton
class Factory:
    def __init__(self):
        self._type = None
        self._products = []

    def create_product(self, type):
        self._products.append(type())

    def show_products(self):
        for p in self._products:
            print(p)
    

class Product:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return self.name
    
    def description(self):
        return f"This is a {self.name} {self.__class__.__name__.capitalize()}"


class Phone(Product):
    def __init__(self, name):
        super().__init__(name)

    
class Laptop(Product):
    def __init__(self, name):
        super().__init__(name)


factory = Factory()
factory.create_product(lambda: Phone("Galaxy 9"))
factory.create_product(lambda: Phone("Pixel 2"))
factory.create_product(lambda: Laptop("Framework 13"))
factory.show_products()
print()

for p in factory._products:
    print(p.description())

Galaxy 9
Pixel 2
Framework 13

This is a Galaxy 9 Phone
This is a Pixel 2 Phone
This is a Framework 13 Laptop


## Ex. 3 — Test **Builder**
- Creează un `LaptopBuilder` cu metode de configurare (ex.: `.cu_ram(int)`, `.cu_ssd(int)`, `.cu_gpu(str)`).
- Metoda finală `.construieste()` întoarce un obiect `Laptop` cu o metodă `descriere()`.
- Construiește un laptop personalizat și afișează descrierea.


In [4]:
class LaptopBuilder(Product):
    def __init__(self, name):
        super().__init__(name)
        self.ram = None
        self.ssd = None
        self.gpu = None

    def with_ram(self, a: int):
        self.ram = a

    def with_ssd(self, a: int):
        self.ssd = a

    def with_gpu(self, a: str):
        self.gpu = a

    def description(self):
        print(f"Laptop contains: {self.ram}GB, {self.ssd}GB, {self.gpu} GPU")
        return super().description()

    def __str__(self):
        return super().__str__()


frame = LaptopBuilder("Framework 13")

frame.with_ram(32)
frame.with_ssd(2000)
frame.with_gpu("discrete iGPU Tesseract")

print(frame)
print(frame.description())

Framework 13
Laptop contains: 32GB, 2000GB, discrete iGPU Tesseract GPU
This is a Framework 13 Laptopbuilder


## Ex. 4 — **Factory** + **Builder** într-un scenariu educațional
- Creează `UtilizatorFactory` care produce tipuri de utilizatori: `Student`, `Trainer`, `Admin` (cu câmpuri de bază, ex.: `nume`).
- Creează `CursBuilder` cu: `.cu_nume(str)`, `.cu_trainer(str)`, `.cu_pret(float)`, `.cu_nivel(str)`, `.construieste()`.
- Folosește `UtilizatorFactory` pentru a crea un `Admin`, apoi construiește un `Curs` și afișează rezultatul.

In [5]:
class User:
    def __init__(self, name: str, password: str, access_level: int):
        self.name = name # hex hash
        self.password = password # hex hash
        self.access_level = access_level # 1-3: 1-Student, 2-Trainer, 3-Admin

class Student(User):
    def __init__(self, name, password):
        super().__init__(name, password, 1)

class Trainer(User):
    def __init__(self, name, password):
        super().__init__(name, password, 2)

class Admin(User):
    def __init__(self, name, password):
        super().__init__(name, password, 3)

class UserFactory:
    @staticmethod
    def make_user(type, name, password):
        if type == "student":
            return Student(name, password)
        if type == "trainer":
            return Trainer(name, password)
        if type == "admin":
            return Admin(name, password)

# cu: `.cu_nume(str)`, `.cu_trainer(str)`, `.cu_pret(float)`, `.cu_nivel(str)`, `.construieste()`.
class CourseBuilder:
    def __init__(self):
        self.name: str = None
        self.trainer: str = None
        self.price: float = None
        self.level: str = None

    def with_name(self, name: str):
        self.name = name

    def with_trainer(self, trainer: str):
        self.trainer = trainer

    def with_price(self, price: float):
        self.price = price

    def with_level(self, level: str):
        self.level = level

    def build_it(self):
        name = input("name: ")
        trainer = input("trainer: ")
        price = input("price: ")
        level = input("level: ")

        self.with_name(name)
        self.with_trainer(trainer)
        self.with_price(price)
        self.with_level(level)

    def __str__(self):
        return(f"name: {self.name}\n"
               f"trainer: {self.trainer}\n"
               f"price: {self.price}\n"
               f"level: {self.level}")

admin = UserFactory.make_user("admin", "admin1", "1234")

course = CourseBuilder()

course.with_name("Python Course")
course.with_trainer("Tonny Raiser")
course.with_price(5000)
course.with_level("Advanced")

print(course)



name: Python Course
trainer: Tonny Raiser
price: 5000
level: Advanced


## Ex. 5 — Mini-quiz recapitulativ (răspunsuri în această celulă sau în cod dacă preferi)
1. Ce problemă rezolvă **Singleton**?
2. **Factory** vs **Builder** — care sunt diferențele de responsabilitate?
3. Ce metode ar trebui să aibă un **Builder** bine structurat?
4. Când ai folosi fiecare pattern dintre cele trei și de ce?


In [6]:
# 1. Single instance, avoid confusion when you need a single Scheduler, Logger, etc...abs
# 2. They look the same to me, classes with purposes. Factory in this example creates a class from different types, 
# builder incapsulates all methods needed to fully construct a more complex object
# 3. A well structured builder should have all necessary  methods to complete the object instantiation within the required scope
# 4. 
# Singleton - Scheduler
#   Why: Avoids accidental multi intantiation
#   Why: Simpler thread safety checks
# The other two factory/builder are really just based on scope, they don't necessarily have a special place

## Ex. 6a — Temă: **Logger Singleton** cu fișier istoric
- Extinde `Logger` ca **Singleton** astfel încât `log()` să scrie mesajele și în fișierul `istoric.txt`.
- Adaugă o opțiune simplă de formatare a mesajelor (ex.: timestamp + nivel).


In [7]:
import datetime

class LoggerNew:
    _instance = None
    _file = "istoric.txt"
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

    def log(self, message: str = None, level: str = "INFO"):
        if message:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            line = f"[{timestamp}] [{level.upper()}] {message}"

            if not line.endswith("\n"):
                line += "\n"

            with open(self._file, 'a') as f:
                f.write(line)

        else:
            with open(self._file, 'r') as f:
                return [line.rstrip('\n') for line in f]
            

logger1 = LoggerNew()

logger1.log("test")
logger1.log("error text", "ERROR")

data = logger1.log()
for l in data:
    print(l)

[2025-10-11 11:38:22] [INFO] test
[2025-10-11 11:38:22] [ERROR] error text
[2025-10-11 11:38:40] [INFO] test
[2025-10-11 11:38:40] [ERROR] error text


## Ex. 6b — Temă: **Factory** de animale
- Implementează o fabrică ce poate returna obiecte `Caine`, `Pisica`, `Vaca`.
- Fiecare clasă expune cel puțin o metodă reprezentativă (ex.: `sunet()` sau `descriere()`).


In [ ]:
class Animal:
    def __init__(self, name):
        self.name = name

    def sound(self):
        return f"{self.__class__.__name__} sound"

    def description(self):
        return f"{self.name} is an animal of type {self.__class__.__name__}"

    def __str__(self):
        return f"[{self.__class__.__name__}] {self.name}"

class Dog(Animal):
    def sound(self): 
        return f"{self.name} barks!"

class Cat(Animal):
    def sound(self): 
        return f"{self.name} meows!"

class Cow(Animal):
    def sound(self): 
        return f"{self.name} moos!"

class AnimalsFactory:
    _instance = None
    _registry = {}

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    @classmethod
    def register(cls, name, animal_cls):
        cls._registry[name.lower()] = animal_cls
    
    def build_animal(self, type_name: str, name: str):
        animal_cls = self._registry.get(type_name.lower())
        if not animal_cls:
            raise ValueError(f"Uknown animal type: {type_name}")
        
        return animal_cls(name)

AnimalsFactory.register("dog", Dog)
AnimalsFactory.register("cat", Cat)
AnimalsFactory.register("cow", Cow)

factory = AnimalsFactory()

dog = factory.build_animal("dog", "Rex")
cat = factory.build_animal("cat", "Misty")
cow = factory.build_animal("cow", "Bessie")

print(dog)            # [Dog] Rex
print(cat)            # [Cat] Misty
print(cow)            # [Cow] Bessie

print(dog.sound())    # Rex sound
print(cat.description())  # Misty is an animal of type Cat

[Dog] Rex
[Cat] Misty
[Cow] Bessie
Rex barks!
Misty is an animal of type Cat


## Ex. 6c — Temă: **Builder** de Pizza
- Creează un `PizzaBuilder` care permite adăugarea de toppinguri (ex.: `.cu_blat(str)`, `.cu_sos(str)`, `.adauga_topping(str)`, `.construieste()`).
- Rezultatul `Pizza` ar trebui să poată fi afișat într-un format prietenos (ex.: `__str__()` / `descriere()`).


In [ ]:
class Pizza:
    def __init__(self, crust, sauce, topping):
        self.crust = crust
        self.sauce = sauce
        self.topping = topping

    def __str__(self):
        msg = "Pizza with "
        msg = msg + f"{self.crust}" if self.crust else msg
        msg = msg + f", {self.sauce}" if self.sauce else msg
        msg = msg + f" and {self.topping}" if self.topping  else msg

        return msg

class PizzaBuilder:  
    def __init__(self):
        self.crust = None
        self.sauce = None
        self.topping = None

    def with_crust(self, crust: str):
        self.crust = crust
        return self

    def with_sauce(self, sauce: str):
        self.sauce = sauce
        return self

    def add_topping(self, topping: str):
        self.topping = topping
        return self
    
    def build(self):
        return Pizza(self.crust, self.sauce, self.topping)
    

builder = PizzaBuilder()

# builder.with_crust("thin crust")
# builder.with_sauce("tomato sauce")
# builder.add_topping("mozzarella")
# pizza = builder.build()

pizza = (builder
          .with_crust("thin crust")
          .with_sauce("tomato sauce")
          .add_topping("mozzarella")
          .build()
          )

print(pizza)

# another pizza
pb = PizzaBuilder()
pb.with_crust("stuffed crust")
pb.with_sauce("white sauce")
pb.add_topping("chicken")
print(pb.build())


Pizza with thin crust, tomato sauce and mozzarella
Pizza with stuffed crust, white sauce and chicken
